# NuSTAR Timing Analysis: CTCV J2056-3014
### *To view plots, visit [nbviewer](https://nbviewer.org/github/ericymiao/ctcvj2056-timing-spin-evolution/blob/main/notebooks/03_nustar_timing.ipynb)*

Period searches for NuSTAR observations.

## NuSTAR-Specific Notes

NuSTAR (Nuclear Spectroscopic Telescope Array) provides focusing hard X-ray observations in the 3-79 keV band, extending well beyond the soft X-ray coverage of NICER and XMM-Newton. Key considerations:

- **Two focal plane modules (FPMA, FPMB)**: Each module has its own optics and detector, providing independent measurements. Both must be analyzed separately to verify consistent detections.
- **Hard X-ray coverage**: The 3-79 keV band probes higher-energy emission from the accretion column and WD surface, complementing the soft X-ray data.
- **Background extraction**: Like XMM, background is extracted from source-free regions on the detector and scaled by extraction area.

In [1]:
import sys
sys.path.insert(0, '../src')

import numpy as np
from stingray import EventList, Lightcurve
from stingray.pulse import z_n_search
from astropy.time import Time, TimeDelta
import hendrics.io as HENio

from bokeh.plotting import figure, show
from bokeh.layouts import column, row, gridplot
from bokeh.io import output_notebook
from bokeh.models import Title

from timing_analysis import (
    fold, get_frequency_uncertainty, bayesian_periodogram
)
from plotting import (
    set_axis_styles, plot_lightcurve, plot_folded_lightcurve, plot_hardness_ratio
)
from statistics import (
    test_hardness_ratio_variability, test_phase_correlation
)

output_notebook()

Loading BokehJS ...

## Load Data

In [16]:
# Load NuSTAR event files
events = {
    'FPMA': {'src': {}, 'bkg': {}},
    'FPMB': {'src': {}, 'bkg': {}},
}

data_path = '../../../nustar/30801006002/event_cl/nu30801006002'

for fpm in ['FPMA', 'FPMB']:
    events[fpm]['src']['total'] = HENio.load_events(f'{data_path}{fpm[-1]}01_cl_bary_src.evt')
    events[fpm]['bkg']['total'] = HENio.load_events(f'{data_path}{fpm[-1]}01_cl_bary_bkg.evt')

# Energy filtering (3-79 keV for NuSTAR)
for fpm in events:
    for region in events[fpm]:
        events[fpm][region]['total'].filter_energy_range([3, 79], inplace=True)
        events[fpm][region]['soft'] = events[fpm][region]['total'].filter_energy_range([3, 10])
        events[fpm][region]['hard'] = events[fpm][region]['total'].filter_energy_range([10, 79])

# Create lightcurves
lightcurves = {}
for fpm in events:
    lightcurves[fpm] = {}
    for region in events[fpm]:
        lightcurves[fpm][region] = {}
        for energy in events[fpm][region]:
            lightcurves[fpm][region][energy] = {}
            for dt in [1, 5, 20, 50, 100]:
                lightcurves[fpm][region][energy][dt] = events[fpm][region][energy].to_lc(dt=dt)

## Observation Summary

In [9]:
print("NuSTAR OBSERVATION SUMMARY\n")

for fpm in events:
    src_counts = len(events[fpm]['src']['total'].time)
    total_time = np.sum(events[fpm]['src']['total'].gti[:, 1] - events[fpm]['src']['total'].gti[:, 0])
    
    print(f"{fpm}:")
    print(f"  Total Time: {total_time/1000:.2f} ks")
    print(f"  Total Counts: {src_counts:,}")
    print()

NuSTAR OBSERVATION SUMMARY

FPMA:
  Total Time: 28.16 ks
  Total Counts: 2,417

FPMB:
  Total Time: 28.18 ks
  Total Counts: 2,287



In [19]:
plots = []
binsize = 100

p = figure(
    width=1100,
    height=350,
    x_axis_label='Time (mjd)',
    y_axis_label='Countrate (cts/s)',
    min_border_bottom=80
)

set_axis_styles(p)
    
for fpm in lightcurves:
    lc = lightcurves[fpm]['src']['total'][binsize]

    color = 'firebrick' if fpm == 'FPMA' else 'tomato'
    plot_lightcurve(p, lc, time_format='utc', color=color, line_width = 1.5)

plots.append(p)

obs_duration = (lc.time[-1]-lc.time[0])
print("Observation length:" ,obs_duration/86400, "days")
print("frequency \"resolution:\"", 1/obs_duration)
    

show(column(plots))

Observation length: 0.6712962962962962963 days
frequency "resolution:" 1.7241379310344827586e-05


## Z^2 Periodogram

Period search in the hard X-ray band. Each focal plane module is analyzed independently to confirm the pulsation detection at high energies.

In [10]:
f_min, f_max = 0.0335, 0.034
frequencies = np.linspace(f_min, f_max, 4096)
nharm = 1

Z2 = {}
plots = []

for fpm in events:
    p = figure(width=600, height=400,
               x_axis_label='Frequency (Hz)',
               y_axis_label='Power',
               title=Title(text=fpm, text_font_size="14pt", align="center"))
    set_axis_styles(p)
    
    evt = events[fpm]['src']['total']
    frequency, power = z_n_search(evt.time, frequencies, nharm=nharm)
    Z2[fpm] = (frequency, power)
    
    p.line(frequency, power, line_width=1.5)
    
    peak_freq = frequencies[np.argmax(power)]
    peak_power = power[np.argmax(power)]
    
    print(f"Module: {fpm}")
    freq_min, freq_max, power_min, _ = get_frequency_uncertainty(
        frequencies, power, peak_freq, peak_power, nharm, 1
    )
    
    p.vspan(x=[freq_min, freq_max], alpha=0.8, line_color='darkviolet')
    p.hspan(power_min, alpha=0.5, line_color='darkviolet')
    
    plots.append(p)

show(row(plots))

Module: FPMA
FREQUENCY RESULTS:
  Peak:        3.377045177045177e-02 Hz
  Uncertainty: +5.982906e-06 / -6.593407e-06 Hz

PERIOD RESULTS:
  Peak:        2.961168558825656e+01 s
  Uncertainty: +5.782570e-03 / -5.245193e-03 s
Module: FPMB
FREQUENCY RESULTS:
  Peak:        3.376959706959707e-02 Hz
  Uncertainty: +5.006105e-06 / -5.128205e-06 Hz

PERIOD RESULTS:
  Peak:        2.961243505331323e+01 s
  Uncertainty: +4.497587e-03 / -4.389184e-03 s


## Folded Profiles

Phase-folded hard X-ray lightcurves in soft (3-10 keV) and hard (10-79 keV) bands. The pulse shape at hard X-rays may differ from soft X-rays if the emission mechanisms or viewing geometry differ with energy.

In [11]:
ephemeris = 58780
P_orb = 29.60968584
n_bins = 20

# Fold lightcurves
folded_lightcurves = {}
for fpm in events:
    folded_lightcurves[fpm] = {}
    for region in events[fpm]:
        folded_lightcurves[fpm][region] = {}
        for energy in events[fpm][region]:
            folded_lightcurves[fpm][region][energy] = fold(
                events[fpm][region][energy], P_orb, ephemeris, n_bins
            )

In [12]:
# Plot folded lightcurves
plots = []

for fpm in folded_lightcurves:
    p = figure(width=500, height=360,
               x_axis_label='Phase',
               y_axis_label='Count Rate (cnts/s)',
               title=Title(text=fpm, text_font_size="13pt", align="center"),
               min_border=20)
    set_axis_styles(p)
    
    for energy in ['soft', 'hard']:
        color = 'firebrick' if energy == 'soft' else 'darkviolet'
        phase, _, cr, err, _ = folded_lightcurves[fpm]['src'][energy]
        plot_folded_lightcurve(p, phase, cr, err, n_bins, color=color)
    
    plots.append(p)

show(row(plots))